# Webscraping Finance

Importer les modules nécessaires :

In [1]:
import requests
import bs4
import pandas as pd
import re
from datetime import datetime
import mysql.connector

Fonction de création de Dataframe en utilisant BeautifulSoup sur Boursorama

In [2]:
def stock_data(stock_name, table_class, index):
    #créer l'URL
    token = "https://www.boursorama.com/bourse/indices/cours/"
    #requête à partir de l'URL
    response = requests.get(str(token + stock_name))
    #BeautifulSoup sur la requête
    soup = bs4.BeautifulSoup(response.text, 'html.parser')
    
    #obtenir le nom du stock en parsant le titre de la page
    stock_name2 = soup.title.string
    stock_name2 = re.split("Cours|,", stock_name2)[0]
    
    #trouver toutes les tables d'une certaine classe
    table_box = soup.find_all("table", {"class": table_class})
    #transformer la liste des tables en liste de dataframes
    data_table = pd.read_html(str(table_box))
    #réduire la liste à la table désirée à partir de l'index donné en argument
    data_table = data_table[index]
    
    #remplacer l'index par la colonne des titres
    data_table.index = data_table[data_table.columns[0]]
    data_table = data_table.drop(data_table.columns[0], axis=1)
    
    #transposer la dataframe pour avoir les titres en colonnes et les dates en index
    data_table = data_table.T
    
    #rajouter une colonne avec le vrai nom de l'indice
    data_table["Indice"] = stock_name2
    
    #rajouter une colonne date
    data_table["Date"] = data_table.index
    
    return data_table

Liste des noms utilisés en URL :

In [3]:
stock_name_list = ["1rPCAC", "1rPPX4", "2zPMS190", "5pDAX", "%24INX", "%24INDU", "2cSX5E", "UKX.L", "FF55-IBEX", "%24COMPX", "1xN225"]

Itération de la fonction créée sur les noms en URL, récupération d'une liste de Dataframes :

In [4]:
stock_list = list()
for stock in stock_name_list:
    stock_df = stock_data(stock, "c-table c-table--generic", -1)
    stock_list.append(stock_df)

Concaténation des Dataframes :

In [5]:
joined_table = pd.concat(stock_list)

joined_table.index = range(len(joined_table))

In [6]:
for index in joined_table.index:
    joined_table["Der."][index] = joined_table["Der."][index].replace(" ", "")
    joined_table["Var."][index] = joined_table["Var."][index].replace("%", "")
    joined_table["Ouv."][index] = str(joined_table["Ouv."][index]).replace(" ", "")
    joined_table["+Haut"][index] = str(joined_table["+Haut"][index]).replace(" ", "")
    joined_table["+Bas"][index] = str(joined_table["+Bas"][index]).replace(" ", "")
    joined_table["Vol."][index] = str(joined_table["Vol."][index]).replace(" ", "")
    day = str(joined_table["Date"][index])[:2]
    month = str(joined_table["Date"][index])[3:]
    french_date = day + "-" + month
    english_date = "2021-" + month + "-" + day
    joined_table["Date"][index] = joined_table["Date"][index].replace(french_date, english_date)
    
joined_table["Der."] = joined_table["Der."].astype(float)
joined_table["Var."] = round((joined_table["Var."].astype(float) / 100), 4)
joined_table["Ouv."] = joined_table["Ouv."].astype(float)
joined_table["+Haut"] = joined_table["+Haut"].astype(float)
joined_table["+Bas"] = joined_table["+Bas"].astype(float)
joined_table["Vol."] = joined_table["Vol."].astype(float)

In [7]:
#joined_table = joined_table.drop(columns=["Vol."])

In [8]:
joined_table = joined_table.rename(columns={"Der.": "Dernier", "Var.": "Variation", "Ouv.": "Ouverture", "+Haut": "Haut", "+Bas": "Bas", "Vol.": "Volume"})

In [9]:
joined_table = joined_table.fillna(0)

In [10]:
display(joined_table)
display(joined_table.dtypes)

Unnamed: 0,Dernier,Variation,Ouverture,Haut,Bas,Volume,Indice,Date
0,6035.970,-0.0017,6069.090,6089.200,6018.550,3434.0,CAC 40,2021-03-15
1,6055.430,0.0032,6050.400,6061.190,6034.760,3792.0,CAC 40,2021-03-16
2,6054.820,-0.0001,6048.310,6062.360,6037.250,3454.0,CAC 40,2021-03-17
3,6062.790,0.0013,6076.210,6082.920,6044.320,3878.0,CAC 40,2021-03-18
4,6010.740,-0.0086,6021.130,6047.650,5983.610,2956.0,CAC 40,2021-03-19
5,4754.330,-0.0019,4780.460,4795.800,4742.480,0.0,SBF 120,2021-03-15
6,4771.780,0.0037,4766.860,4777.560,4757.280,0.0,SBF 120,2021-03-16
7,4771.370,-0.0001,4766.680,4775.730,4756.420,0.0,SBF 120,2021-03-17
8,4775.180,0.0008,4789.490,4795.500,4763.440,0.0,SBF 120,2021-03-18
9,4736.700,-0.0081,4743.480,4762.780,4714.320,0.0,SBF 120,2021-03-19


Unnamed: 0
Dernier      float64
Variation    float64
Ouverture    float64
Haut         float64
Bas          float64
Volume       float64
Indice        object
Date          object
dtype: object

Sauvegarde en fichier .csv :

In [11]:
joined_table.to_csv("boursorama.csv", sep=";", index=False)

In [12]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root",
  port="3307",
  database="boursorama"
)

InterfaceError: 2013: Lost connection to MySQL server during query

In [ ]:
mycursor = mydb.cursor()

In [ ]:
for index in joined_table.index:
    sql = "INSERT INTO cours (Dernier, Variation, Ouverture, Haut, Bas, Volume, Indice, Date) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
    val = (float(joined_table["Dernier"].iloc[index]), float(joined_table["Variation"].iloc[index]), float(joined_table["Ouverture"].iloc[index]), float(joined_table["Haut"].iloc[index]), float(joined_table["Bas"].iloc[index]), float(joined_table["Volume"].iloc[index]), str(joined_table["Indice"].iloc[index]), joined_table["Date"].iloc[index])
    mycursor.execute(sql, val)
    mydb.commit()
